In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
import yaml
import os



class Answer(BaseModel):
    title_en: str = Field(description="")
    title_ko: str = Field(description="")
    contents: str = Field(default="")
    detail_contents: str = Field(description="")
    urls: str = Field(description="")

class QandAChain:
    def __init__(self):
        base_path = os.getcwd() 
        # base_path = os.path.dirna/e(os.path.abspath(__file__))  # 현재 파일의 경로
        file_path = os.path.join(base_path, 'prompt', 'QandAPrompt.yaml')

        #with open(file_path, 'r') as f:
        #    self.__prompt_template = yaml.load(f, Loader=yaml.SafeLoader)['template']
        
        self.__prompt_template = """
        너는 여행 정보를 제공하는 AI야.

        사용자의 질문에 아래 JSON 형식으로 답변해 줘:

        {format_instructions}

        질문: {question}
        """

        self.__model = ChatOpenAI(
            temperature = 0.1,
            model_name="gpt-4o-mini",
            max_tokens = 2048,
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        self.__parser = JsonOutputParser(pydantic_object=Answer)
        # self.__parser = StrOutputParser()

    def main_chain_invoke(self, question):
        prompt = PromptTemplate.from_template(self.__prompt_template)
        prompt = prompt.partial(format_instructions=self.__parser.get_format_instructions())

        chain = prompt | self.__model | self.__parser

        result = chain.invoke({"question": question})
        return result

In [13]:
if __name__ == "__main__":
    chain = QandAChain()
    result = chain.main_chain_invoke("LangChain이란?")
    print(result)

{'title_en': 'What is LangChain?', 'title_ko': 'LangChain이란?', 'contents': 'LangChain은 자연어 처리(NLP) 및 인공지능(AI) 모델을 활용하여 다양한 애플리케이션을 구축할 수 있도록 돕는 프레임워크입니다.', 'detail_contents': 'LangChain은 언어 모델을 사용하여 데이터 처리, 대화형 AI, 정보 검색, 자동화된 작업 등을 수행할 수 있는 도구와 라이브러리를 제공합니다. 이 프레임워크는 사용자가 쉽게 언어 모델을 통합하고, 다양한 데이터 소스와 상호작용할 수 있도록 설계되었습니다.', 'urls': 'https://langchain.readthedocs.io/'}


In [ ]:
# import os
# import requests
# from dotenv import load_dotenv
# # import DB.db_connect
# import DB.SqlModel.Postgresql.CityRegion from city_region
# import DB.SqlModel.Postgresql.ServiceCategory from service_category
# import CRUD.select from select

# load_dotenv()
# service_key = os.getenv("TOURAPI_SERVICE_KEY")

# question = "강릉에서 갈만한 관광지 알려줘."
# best_match = find_similar_parameters(question, similarity_threshold=0.7)

# if not best_match:
#     print("관련된 지역/카테고리를 찾을 수 없습니다.")
#     exit()

# url = "https://apis.data.go.kr/B551011/KorService2/areaBasedList2"

# params = {
#     "MobileOS" : "WEB",
#     "MobileApp" : "TOUR_AI",
#     "serviceKey": self.__service_key,
#     "lDongRegnCd" : lDongRegnCd,
#     "lDongSignguCd" : lDongSignguCd,
#     "contentTypeId": contentTypeId,
#     "cat1": cat1,
#     "cat2": cat2,
#     "cat3": cat3
# }

# response = requests.get(url, params=params)

# if response.status_code == 200:
#     data = response.json()  
#     items = data.get('response', {}).get('body', {}).get('items', {}).get('item', [])

#     for i, item in enumerate(items, 1):
#         print(item)
#         # break
#         title = item.get('title', '제목없음')
#         addr1 = item.get('addr1', '주소없음')
#         tel = item.get('tel', '전화번호없음')
#         mapx = item.get('mapx', '')
#         mapy = item.get('mapy', '')
#         print(f"{i}. {title}")
#         print(f"   주소: {addr1}")
#         print(f"   전화: {tel}")
#         print(f"   좌표: ({mapx}, {mapy})")
#         print()
#     # print(data)
# else:
#     print(f"Error: {response.status_code}")


SyntaxError: Did you mean to use 'from ... import ...' instead? (1364386622.py, line 5)

In [ ]:
%cd ~/teamproj/ai_chatbot_travel_booking/AI/app/
from openai import OpenAI
import os
from CRUD.select import SELECT_DB
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

QUESTION = "강릉의 관광명소를 알려줘"

q_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=QUESTION
).data[0].embedding

# 2. DB에서 유사도 검색
selector = SELECT_DB()
similar = selector.get_similar_city_region(q_embedding, top_k=3)

# 3. 결과 출력
for row in similar:
    print(row.city_str, row.city_code1, row.city_code2)


/home/jb01/teamproj/ai_chatbot_travel_booking/AI/app


NameError: name 'dbconnect' is not defined

In [ ]:
%cd ~/teamproj/ai_chatbot_travel_booking/AI/app/
from openai import OpenAI
import os
from CRUD.select import SELECT_DB
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

QUESTION = "강릉의 관광명소를 알려줘"

q_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=QUESTION
).data[0].embedding

print(f"임베딩 길이: {len(q_embedding)}")

# DB에서 유사도 검색
selector = SELECT_DB()
similar = selector.get_similar_city_region(q_embedding, top_k=3)

# 결과 출력
for row in similar:
    print(f"도시: {row[2]}")  # city_str
    print(f"코드: {row[0]}-{row[1]}")  # city_code1, city_code2
    if len(row) > 3:
        print(f"거리: {row[3]:.4f}")  # distance
    print("---")

/home/jb01/teamproj/ai_chatbot_travel_booking/AI/app
임베딩 길이: 1536


ProgrammingError: (psycopg2.errors.UndefinedFunction) operator does not exist: vector <-> numeric[]
LINE 3:                        embedding <-> ARRAY[ -0.0312573984265...
                                         ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: 
                SELECT city_code1, city_code2, city_str, 
                       embedding <-> %(embedding_param)s as distance
                FROM city_regions 
                ORDER BY embedding <-> %(embedding_param)s
                LIMIT %(limit_param)s
            ]
[parameters: {'embedding_param': [-0.031257398426532745, -0.0372454859316349, -0.0005025439895689487, 0.0016642147675156593, -0.02819133549928665, -0.012490613386034966, -0.0681530684 ... (33917 characters truncated) ... 684476613998413, 0.004179828334599733, 0.015145125798881054, 0.019198916852474213, -0.026339350268244743, -0.03092815913259983, -0.013344584964215755], 'limit_param': 3}]
(Backgr